In [1]:
import os

# Local and FTP addresses 
DATA_ROOT = './data/'
RAW_DATA_PATH = './data/raw/'
PROCESSED_DATA_PATH = './data/processed/'

PCBA_CID_FILE_NAME = 'Cid2BioactivityLink'
CID_INCHI_FILE_NAME = 'CID-InChI-Key'

PCBA_CID_FTP_ADDRESS = 'ftp://ftp.ncbi.nlm.nih.gov/pubchem/Bioassay/Extras/%s.gz' % PCBA_CID_FILE_NAME
CID_INCHI_FTP_ADDRESS = 'ftp://ftp.ncbi.nlm.nih.gov/pubchem/Compound/Extras/%s.gz' % CID_INCHI_FILE_NAME

PCBA_CID_FILE_PATH = os.path.join(RAW_DATA_PATH, PCBA_CID_FILE_NAME)
CID_INCHI_FILE_PATH = os.path.join(RAW_DATA_PATH, CID_INCHI_FILE_NAME)

# Download and unpack data 
if not os.path.exists(PCBA_CID_FILE_PATH):
    os.system('wget -r -nd -nc %s -P %s' % (PCBA_CID_FTP_ADDRESS, RAW_DATA_PATH))
    os.system('find %s -type f -iname \"*.gz\" -exec gunzip {} +' % RAW_DATA_PATH)

if not os.path.exists(CID_INCHI_FILE_PATH):
    os.system('wget -r -nd -nc %s -P %s'  % (CID_INCHI_FTP_ADDRESS, RAW_DATA_PATH))
    os.system('find %s -type f -iname \"*.gz\" -exec gunzip {} +' % RAW_DATA_PATH)


In [20]:
import json
import pandas as pd
from rdkit import Chem
try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle

CHUNK_SIZE = 2 ** 16

# TODO: This is a very stupid way for trying to locate the chunk of a CID
# Note that the CID is strictly incremental, which we can take advantage of. 
CID_MOL_FILE_NAME = './CID-Mol/Mol/CID-mol_%04i.pkl'
CID_MOL_FILE_PATH = os.path.join(PROCESSED_DATA_PATH, CID_MOL_FILE_NAME)

# Need to specify the FP (probably using ECFP4(1024) + ECFP6(1024))
# CID_ECFP_FILE_NAME
# CID_ECFP_FILE_PATH

# Need to decide either using sparse matrix of dense one
# CID_GRAPH_FILE_NAME
# CID_GRAPH_FILE_NAME

CHUNK_CID_FILE_NAME = './CID-Mol/chunk_num-CID.txt'
CHUNK_CID_FILE_PATH = os.path.join(PROCESSED_DATA_PATH, CHUNK_CID_FILE_NAME)
UNUSED_CID_FILE_NAME = './CID-Mol/unused_CID.txt'
UNUSED_CID_FILE_PATH = os.path.join(PROCESSED_DATA_PATH, UNUSED_CID_FILE_NAME)

# cid_mol = [] 
unused_cid = []
chunk_num_cid_dict = {}

for chunk_idx, cid_inchi_df_chunk in enumerate(
        pd.read_csv(CID_INCHI_FILE_PATH,
                    sep='\t',
                    header=None,
                    index_col=[0],
                    usecols=[0, 1],
                    chunksize=CHUNK_SIZE)):
    
    cid_mol_chunk = [] 
    for cid, row in cid_inchi_df_chunk.iterrows():
        
        inchi = row[1]
        try:
            mol = Chem.MolFromInchi(inchi)
            assert mol
            
            # Extract the following features:
            #  * SMILES, 
            #  * fingerprint
            #  * graph (nodes and sparse adjacency matrix)
            
            # SMILES
            # (do we encode the SMILES here or compute on the fly?)
            # smiles = Chem.MolToSmiles()
            # assert len(smiles) != 0
            
            # Fingerprint 
            # # Encoding to hex bit strings
            # bit_str = AllChem.GetMorganFingerprintAsBitVect(
            #         mol, radius=radius, nBits=FP_N_BITS,
            #         useFeatures=useFeatures).ToBitString()
            # hex_str = hex(int(bitStr, 2))
            # 
            # # Decode to feature (list of 0, 1)
            # bit_str = str(bin(int(hex_str, 16)))[2:]
            # feature = np.concatenate(
            #     (np.zeros(shape=(1024 - len(bit_str))),
            #      np.array([np.int(c) for c in bit_str])), axis=0)
            
            # TODO
            # Graph 
            # (do we encode the graph here or compute on the fly?)
            
            # Will only save this molecule and its features if all the 
            # features are valid. 
            
        except AssertionError:
            # print('Failed converting compoud (CID=%i) to molecule.' % cid)
            unused_cid.append(cid)
            continue
        
        if chunk_idx in chunk_num_cid_dict:
            chunk_num_cid_dict[chunk_idx].append(cid)
        else:
            chunk_num_cid_dict[chunk_idx] = [cid, ]
        
        cid_mol_chunk.append(row)

    with open(CID_MOL_FILE_PATH % chunk_idx, 'w+b') as f:
        pickle.dump(cid_mol_chunk, f, pickle.HIGHEST_PROTOCOL)

with open(CHUNK_CID_FILE_PATH, 'w') as f:
    json.dump(chunk_num_cid_dict, f, indent=4)
with open(UNUSED_CID_FILE_PATH, 'w') as f:
    json.dump(unused_cid, f, indent=4)
